In [83]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [84]:
tf.__version__

'1.11.0'

In [85]:
# get data through tensorflow MNIST tutorials Data.
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST/', one_hot=True)

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


In [86]:
data.train.cls = np.argmax(data.train.labels, axis = 1)
data.test.cls = np.argmax(data.test.labels, axis = 1)
data.train.labels[0:10] #setting 10 labels to test later

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [87]:
# setting paramters

img_size = 28
img_size_flat = img_size * img_size 
img_shape = (img_size, img_size)
num_channels = 1
num_classes = 10

Rather than providing raw data directly to the Estimator, we must provide functions that return the data. This allows for more flexibility in data-sources and how the dta is randomly shutffled and iterated.

Note that we will create an Estimator with DNNClassifier, which assumes the class-numbers are integers so we use data.train.cls instead of data.train.labels which are one-hot encoded arrays.

The function also had paramters for batch_size, que_capacity and num_threads for finer control of the data reading. In our case we take the data directly from a numpy array in memory, so it is not needed.

In [88]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(data.train.images)},
    y=np.array(data.train.cls),
    num_epochs=None,
    shuffle=True)

In [89]:
# Calling this function returns a tuple with tensorflow ops for returning the input and output data:
train_input_fn()

({'x': <tf.Tensor 'random_shuffle_queue_DequeueMany_2:1' shape=(128, 784) dtype=float32>},
 <tf.Tensor 'random_shuffle_queue_DequeueMany_2:2' shape=(128,) dtype=int64>)

In [90]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(data.test.images)},
    y=np.array(data.test.cls),
    num_epochs=1,
    shuffle=False)

In [91]:
some_images = data.test.images[0:9] # to test few images

In [92]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": some_images},
    num_epochs=1,
    shuffle=False)

In [93]:
some_images_cls = data.test.cls[0:9]

In [94]:
#using DNN premade classifier, let's do feature columns.
feature_x = tf.feature_column.numeric_column("x", shape=img_shape)

In [95]:
feature_columns = [feature_x]

In [96]:
#use 3-layer DNN with 512,256,128 units respectively
num_hidden_units = [512, 256, 128]

The DNNClassifier constructs the NN for us. We can also specify the activation function and various other parameters. Here, I will use relu for activation function and just specify the number of classes and the directory where the checkpionts will be saved.

In [97]:
model = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                  hidden_units=num_hidden_units,
                                  activation_fn=tf.nn.relu,
                                  n_classes=num_classes,
                                  model_dir="./checkpoints_1")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './checkpoints_1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3c1ffa80f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [98]:
#Train

# Train the model for a given number of iterations. It automatically loads and saves checkpoints so we can continue training later.

model.train(input_fn=train_input_fn, steps=2000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./checkpoints_1/model.ckpt-6000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 6000 into ./checkpoints_1/model.ckpt.
INFO:tensorflow:loss = 1.3127608, step = 6001
INFO:tensorflow:global_step/sec: 129.426
INFO:tensorflow:loss = 3.2103305, step = 6101 (0.774 sec)
INFO:tensorflow:global_step/sec: 135.529
INFO:tensorflow:loss = 7.220837, step = 6201 (0.738 sec)
INFO:tensorflow:global_step/sec: 136.715
INFO:tensorflow:loss = 0.8946363, step = 6301 (0.732 sec)
INFO:tensorflow:global_step/sec: 144.42
INFO:tensorflow:loss = 0.34532276, step = 6401 (0.692 sec)
INFO:tensorflow:global_step/sec: 146.473
INFO:tensorflow:loss = 2.1448255, step = 6501 (0.683 sec)
INFO:tensorflow:global_step/sec: 145.221
INFO:tensorflow:loss = 0.282827

In [99]:
#Evalution

result = model.evaluate(input_fn = test_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-27-20:34:32
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./checkpoints_1/model.ckpt-8000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-27-20:34:32
INFO:tensorflow:Saving dict for global step 8000: accuracy = 0.9731, average_loss = 0.13056402, global_step = 8000, loss = 16.52709
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 8000: ./checkpoints_1/model.ckpt-8000


In [100]:
#result

result

{'accuracy': 0.9731,
 'average_loss': 0.13056402,
 'loss': 16.52709,
 'global_step': 8000}

In [172]:
#predictions

# note that the tensorflow graph is recreated and the checkpoint is reloaded everytime we make predictions on new data.
# if the model is very large then this could add a significant overhead.

# this method will not return the entire prediction dataset at once but it returns a python generator 
# which we can use to iterate through the predictions one by one
# first we initialize our generator

predictions = model.predict(input_fn=predict_input_fn)

In [173]:
pred = np.array(list(predictions))
cls_pred_1 = [pred[i]['classes'] for i in range(len(pred))]
cls_pred_1

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./checkpoints_1/model.ckpt-8000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[array([b'7'], dtype=object),
 array([b'2'], dtype=object),
 array([b'1'], dtype=object),
 array([b'0'], dtype=object),
 array([b'4'], dtype=object),
 array([b'1'], dtype=object),
 array([b'4'], dtype=object),
 array([b'9'], dtype=object),
 array([b'5'], dtype=object)]

In [174]:
l=np.int(str(list(cls_pred[0])[0])[2])

In [191]:
cls_pred_2 = [np.int(str(list(cls_pred[i])[0])[2]) for i in range(9)]

In [192]:
cls_pred_2

[7, 2, 1, 0, 4, 1, 4, 9, 5]